# My Pipeline

I started with data gathered by reddit user u/timtlm and the owner/creator of [Splat Meta](https://www.splatmeta.ink). He has openly released this data and it is free to use.

Here is how he got the data:
* Data is gathered from SplatNet API accessible via the mobile app.
* Every two hours Nintendo updates the top 100 League Teams and Pairs.
* Data is retreieved from the API every 2 hours to get all weapon usages.
* Every week a new table is created summarizing the weapon usage for the previous week.

I have chosen to use this data because the API is designed to be accessed from a mobile app and is diffulct to access from a computer. In addition, it would be impossible to get timeseries data since I do not have the time to gather data from the API for weeks. These weekly summaries are much more limited than raw data, but as a tradeoff it allows me to work with a much larger timeframe.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

## A function to read my csvs, devoloped through EDA detailed below

In [2]:
def splatoon_csv(filename, n, skip = 1, percent = False):
    out = pd.read_csv('data/raw/' + filename, skiprows = skip)
    
    # set name of distribution based on week starting 10/28 to 11/3
    name = 'Week ' + str(n)
    
    # turn percent into a distribution, 2 different naming conventions
    if percent:
        out[name] = out['Percent']/100
    else:
        out[name] = out['Pct']/100
    
    # pick only the collumns I need
    return out[['Weapon Name', name]]

## Initial look

In [3]:
# use bash to get a list of all the filenames in my data folder
filenames = !ls data/raw
filenames

['Week 1 10_28-11_3.csv',
 'Week 10 12_30-1_5.csv',
 'Week 11 1_6-1_12.csv',
 'Week 12 1_13-1_19.csv',
 'Week 13 1_20-1_26.csv',
 'Week 14 1_27-2_2.csv',
 'Week 15 2_3-2_9.csv',
 'Week 16 2_10-2_16.csv',
 'Week 17 2_17-2_23.csv',
 'Week 18 2_24-3_2.csv',
 'Week 19 3_3-3_9.csv',
 'Week 2 11_4-11_10.csv',
 'Week 20 3_10-3_16.csv',
 'Week 21 3_17-3_23.csv',
 'Week 22 3_24-3_30.csv',
 'Week 23 3_31-4_6.csv',
 'Week 24 4_7-4_13.csv',
 'Week 25 4_14-4_20.csv',
 'Week 26 4_21-4_27.csv',
 'Week 27 4_28-5_4.csv',
 'Week 28 5_5-5_11.csv',
 'Week 29 5_12-5_18.csv',
 'Week 3 11_11-11_17.csv',
 'Week 30 5_19-5_25.csv',
 'Week 31 5_26-6_1.csv',
 'Week 4 11_18-11_24.csv',
 'Week 5 11_25-12_1.csv',
 'Week 6 12_2-12_8.csv',
 'Week 7 12_9-12_15.csv',
 'Week 8 12_16-12_22.csv',
 'Week 9 12_23-12_29.csv']

In [4]:
# a function from stackexchange to sort numbers in strings
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [5]:
# sort list
filenames = sorted(filenames, key=natural_keys)
filenames[:5]

['Week 1 10_28-11_3.csv',
 'Week 2 11_4-11_10.csv',
 'Week 3 11_11-11_17.csv',
 'Week 4 11_18-11_24.csv',
 'Week 5 11_25-12_1.csv']

In [6]:
# looked at the head of all files
for file in filenames:
    display(pd.read_csv('data/raw/'+str(file)).head())

,Weapon Name,Count,Percent
0,Splattershot,4978,17.10
1,Range Blaster,1989,6.83
2,Splat Roller,1954,6.71
3,N-ZAP '85,1790,6.15
4,Splat Brella,1438,4.94


,Weapon Name,Count,Percent
0,Splattershot,4573,15.98
1,Splat Roller,1982,6.93
2,N-ZAP '85,1902,6.65
3,Range Blaster,1828,6.39
4,Splat Brella,1668,5.83


,Weapon Name,Count,Percent
0,Splattershot,3880,15.48
1,Splat Roller,1893,7.55
2,N-ZAP '85,1789,7.14
3,Range Blaster,1546,6.17
4,Splat Brella,1482,5.91


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Totals do not add up: Missing some league schedule data so cannot identify game mode for 11/19 18:00 - 11/20 14:00 (UTC),Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,All Modes,NaN,NaN,NaN,NaN,Rainmaker,NaN,NaN,NaN,Splat Zones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Weapon Name,Rank,Count,Percent,NaN,Weapon Name,Count,Percent,NaN,Weapon Name,...,Count,Percent,NaN,Special,Count,Pct,NaN,Sub,Count,Pct
2,Splattershot,1,4238,14.62,NaN,Splattershot,1209,14.72,NaN,Splattershot,...,1203,14.17,NaN,Splashdown,7523,25.96,NaN,Burst Bomb,5423,18.71
3,Splat Roller,2,2064,7.12,NaN,Splat Roller,661,8.05,NaN,Splat Brella,...,641,7.55,NaN,Ink Storm,5854,20.2,NaN,Suction Bomb,4865,16.79
4,N-ZAP '85,3,2008,6.93,NaN,N-ZAP '85,572,6.97,NaN,Range Blaster,...,615,7.24,NaN,Sting Ray,4788,16.52,NaN,Splat Bomb,4634,15.99


,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Special,Unnamed: 29,Unnamed: 30,Unnamed: 31,Sub,Unnamed: 33,Unnamed: 34
0,Weapon Name,NaN,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff,NaN,NaN,...,NaN,Diff,NaN,Name,Count,Pct,NaN,Name,Count,Pct
1,Splattershot,1.0,2784,9.41,14.62,-5.21,1,0,NaN,* 1) Splattershot (9.41) -5.21,...,1.0,0,NaN,Splashdown,6654,22.5,NaN,Splat Bomb,5462,18.47
2,N-ZAP '85,2.0,2294,7.76,6.93,0.83,3,1,NaN,* 2) N-ZAP '85 (7.76) 0.83,...,2.0,0,NaN,Ink Storm,5441,18.4,NaN,Suction Bomb,5179,17.51
3,Splat Roller,3.0,2195,7.42,7.12,0.3,2,-1,NaN,* 3) Splat Roller (7.42) 0.3,...,3.0,0,NaN,Ink Armor,4480,15.15,NaN,Burst Bomb,4551,15.39
4,Tentatek Splattershot,4.0,1935,6.54,3.59,2.95,7,3,NaN,* 4) Tentatek Splattershot (6.54) 2.95,...,4.0,0,NaN,Sting Ray,4223,14.28,NaN,Curling Bomb,3715,12.56


...

,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Weapon Name,Rank,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff,NaN,NaN
1,N-ZAP '85,1,1619,5.96,6.351488498,-0.39,1,0,NaN,1) N-ZAP '85 (5.96) -0.39
2,Tentatek Splattershot,2,1452,5.34,3.332205683,2.01,6,4,NaN,2) Tentatek Splattershot (5.34) 2.01
3,Enperry Splat Dualies,3,1321,4.86,3.915764547,0.95,5,2,NaN,3) Enperry Splat Dualies (4.86) 0.95
4,Splat Roller,4,1269,4.67,5.928619756,-1.26,2,-2,NaN,4) Splat Roller (4.67) -1.26


,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Weapon Name,Rank,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff
1,N-ZAP '85,1,1619,6.42,5.96,0.46,1,0
2,Tentatek Splattershot,2,1377,5.46,5.34,0.12,2,0
3,Splat Roller,3,1259,4.99,4.67,0.32,4,1
4,Enperry Splat Dualies,4,1189,4.72,4.86,-0.15,3,-1


,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Weapon Name,Rank,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff
1,N-ZAP '85,1,1661,6.53,6.42,0.11,1,0
2,Tentatek Splattershot,2,1420,5.58,5.46,0.12,2,0
3,Splat Roller,3,1288,5.06,4.99,0.07,3,0
4,Enperry Splat Dualies,4,1216,4.78,4.72,0.06,4,0


,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Weapon Name,Rank,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff
1,N-ZAP '85,1,1431,6.63,6.53,0.10,1,0
2,Tentatek Splattershot,2,1266,5.87,5.58,0.29,2,0
3,Enperry Splat Dualies,3,1108,5.13,4.78,0.36,4,1
4,Splat Roller,4,1001,4.64,5.06,-0.42,3,-1


,All Modes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Weapon Name,Rank,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff,NaN,NaN
1,N-ZAP '85,1,1426,6.19,6.63,-0.44,1,0,NaN,1) N-ZAP '85 (6.19) -0.44
2,Tentatek Splattershot,2,1352,5.87,5.87,0.00,2,0,NaN,2) Tentatek Splattershot (5.87) 0.00
3,Enperry Splat Dualies,3,1112,4.83,5.13,-0.31,3,0,NaN,3) Enperry Splat Dualies (4.83) -0.31
4,Splat Roller,4,1107,4.81,4.64,0.17,4,0,NaN,4) Splat Roller (4.81) 0.17


## Format differences

In [7]:
# week 1 through 3 is pretty sparse, easy to work with
test = pd.read_csv('data/raw/Week 1 10_28-11_3.csv')
test.head()

,Weapon Name,Count,Percent
0,Splattershot,4978,17.10
1,Range Blaster,1989,6.83
2,Splat Roller,1954,6.71
3,N-ZAP '85,1790,6.15
4,Splat Brella,1438,4.94


In [8]:
# week 4 has 2 unnecessary header rows
test = pd.read_csv('data/raw/Week 4 11_18-11_24.csv', skiprows=2)
test.head()

,Weapon Name,Rank,Count,Percent,Unnamed: 4,Weapon Name.1,Count.1,Percent.1,Unnamed: 8,Weapon Name.2,...,Count.3,Percent.3,Unnamed: 16,Special,Count.4,Pct,Unnamed: 20,Sub,Count.5,Pct.1
0,Splattershot,1,4238,14.62,NaN,Splattershot,1209,14.72,NaN,Splattershot,...,1203,14.17,NaN,Splashdown,7523.0,25.96,NaN,Burst Bomb,5423.0,18.71
1,Splat Roller,2,2064,7.12,NaN,Splat Roller,661,8.05,NaN,Splat Brella,...,641,7.55,NaN,Ink Storm,5854.0,20.20,NaN,Suction Bomb,4865.0,16.79
2,N-ZAP '85,3,2008,6.93,NaN,N-ZAP '85,572,6.97,NaN,Range Blaster,...,615,7.24,NaN,Sting Ray,4788.0,16.52,NaN,Splat Bomb,4634.0,15.99
3,Range Blaster,4,1836,6.34,NaN,Range Blaster,432,5.26,NaN,N-ZAP '85,...,593,6.98,NaN,Ink Armor,3647.0,12.58,NaN,Curling Bomb,3556.0,12.27
4,Splat Brella,5,1782,6.15,NaN,Splat Brella,398,4.85,NaN,Splat Roller,...,513,6.04,NaN,Inkjet,2024.0,6.98,NaN,Sprinkler,2885.0,9.96


In [9]:
#all files from week 5 have 1 unnecessary header row and 'pct' instead of 'Percent'
test = pd.read_csv('data/raw/Week 5 11_25-12_1.csv', skiprows=1)
test.head()

,Weapon Name,Unnamed: 1,Count,Pct,Previous Pct,Pct Diff,Previous Rank,Rank Diff,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Diff.2,Unnamed: 27,Name,Count.4,Pct.4,Unnamed: 31,Name.1,Count.5,Pct.5
0,Splattershot,1,2784,9.41,14.62,-5.21,1.0,0.0,NaN,* 1) Splattershot (9.41) -5.21,...,1,0,NaN,Splashdown,6654.0,22.50,NaN,Splat Bomb,5462.0,18.47
1,N-ZAP '85,2,2294,7.76,6.93,0.83,3.0,1.0,NaN,* 2) N-ZAP '85 (7.76) 0.83,...,2,0,NaN,Ink Storm,5441.0,18.40,NaN,Suction Bomb,5179.0,17.51
2,Splat Roller,3,2195,7.42,7.12,0.30,2.0,-1.0,NaN,* 3) Splat Roller (7.42) 0.3,...,3,0,NaN,Ink Armor,4480.0,15.15,NaN,Burst Bomb,4551.0,15.39
3,Tentatek Splattershot,4,1935,6.54,3.59,2.95,7.0,3.0,NaN,* 4) Tentatek Splattershot (6.54) 2.95,...,4,0,NaN,Sting Ray,4223.0,14.28,NaN,Curling Bomb,3715.0,12.56
4,Range Blaster,5,1707,5.77,6.34,-0.57,4.0,-1.0,NaN,* 5) Range Blaster (5.77) -0.57,...,5,0,NaN,Inkjet,3420.0,11.56,NaN,Autobomb,2948.0,9.97


## Testing my function with the 3 different types of data

In [10]:
df1 = splatoon_csv('Week 1 10_28-11_3.csv', 1, 0, True)
df1.head()

,Weapon Name,Week 1
0,Splattershot,0.1710
1,Range Blaster,0.0683
2,Splat Roller,0.0671
3,N-ZAP '85,0.0615
4,Splat Brella,0.0494


In [11]:
df4 = splatoon_csv('Week 4 11_18-11_24.csv', 4, 2, True)
df4.head()

,Weapon Name,Week 4
0,Splattershot,0.1462
1,Splat Roller,0.0712
2,N-ZAP '85,0.0693
3,Range Blaster,0.0634
4,Splat Brella,0.0615


In [12]:
df5 = splatoon_csv('Week 5 11_25-12_1.csv', 5)
df5.head()

,Weapon Name,Week 5
0,Splattershot,0.0941
1,N-ZAP '85,0.0776
2,Splat Roller,0.0742
3,Tentatek Splattershot,0.0654
4,Range Blaster,0.0577


## Bringing it together

In [13]:
# a loop to join all csvs
for n, name in enumerate(filenames, start=1):
    
    # read list based on 3 possible conditions, hardcoded due to specifity of this case
    if n < 4:
        current_df = splatoon_csv(name, n, 0, True)
    elif n == 4:
        current_df = splatoon_csv(name, n, 2, True)    
    else:
        current_df = splatoon_csv(name, n)
        
    # outer join each df with the previous
    if n == 1:
        out_df = current_df
    else:
        out_df = out_df.join(current_df.set_index('Weapon Name'), on='Weapon Name', how='outer')
    
out_df.head()

,Weapon Name,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,...,Week 22,Week 23,Week 24,Week 25,Week 26,Week 27,Week 28,Week 29,Week 30,Week 31
0,Splattershot,0.1710,0.1598,0.1548,0.1462,0.0941,0.0940,0.0852,0.0887,0.0858,...,0.0652,0.0652,0.0563,0.0535,0.0461,0.0429,0.0444,0.0422,0.0372,0.0355
1,Range Blaster,0.0683,0.0639,0.0617,0.0634,0.0577,0.0541,0.0396,0.0324,0.0298,...,0.0203,0.0232,0.0208,0.0198,0.0151,0.0183,0.0155,0.0167,0.0133,0.0130
2,Splat Roller,0.0671,0.0693,0.0755,0.0712,0.0742,0.0759,0.0766,0.0802,0.0827,...,0.0613,0.0656,0.0650,0.0644,0.0593,0.0467,0.0499,0.0506,0.0464,0.0481
3,N-ZAP '85,0.0615,0.0665,0.0714,0.0693,0.0776,0.1217,0.1369,0.1290,0.1301,...,0.0738,0.0713,0.0785,0.0771,0.0635,0.0596,0.0642,0.0653,0.0663,0.0619
4,Splat Brella,0.0494,0.0583,0.0591,0.0615,0.0570,0.0529,0.0312,0.0220,0.0199,...,0.0207,0.0128,0.0123,0.0122,0.0136,0.0116,0.0102,0.0100,0.0127,0.0119


I do not mind missing values because new weapons were introduced through the lifetime of the game, the NaN simply means that the weapon was introduced after that time and thus has no usage.

In [14]:
# save result as csv
out_df.to_csv('data/usage_timeseries.csv')